## Preparation

In [1]:
import pandas as pd
import numpy as np

In [2]:

%reload_ext sql
%sql duckdb://

## 修改查询结果展示条数

In [38]:
%config SqlMagic.displaylimit = None

displaylimit: Value None will be treated as 0 (no limit)

In [ ]:

%%sql
select id, name, stargazers, forks, releases
  from ghgolang1.csv
 where isFork = 'FALSE'
 order by stargazers desc, forks desc
 limit 100


### Find Earliest Golang Repository

In [ ]:
%%sql
select id, name, stargazers, forks, releases, createdAt
  from ghgolang1.csv
 order by createdAt desc, stargazers desc, forks desc
 limit 100


In [10]:
%%sql
select count(id)
  from ghgolang1.csv


Running query in 'duckdb://'

count(id)


In [ ]:
%%sql
select t2.full_name   as github_name,
       t1.dep_module  as lib_public_name,
       t1.dep_version as lib_version,
       t1.full_name   as ref_name,
       t1.version     as ref_version
  from read_csv_auto('dependencies2.csv', header=True) t1
 inner join read_csv_auto('dependencies2.csv', header=True) t2
    on t1.dep_module = t2.public_name and t2.fork = 0
 order by 1
  limit 1000



In [ ]:
%%sql
select fork,
       count(*)
  from read_csv_auto('repo-candidates/slim.csv', header=True)
  group by fork



In [ ]:
%%sql
select distinct t2.full_name   as github_name,
       t1.dep_module  as lib_public_name,
       t1.full_name   as ref_name,
  from read_csv_auto('dependencies2.csv', header=True) t1
 inner join read_csv_auto('dependencies2.csv', header=True) t2
    on t1.dep_module = t2.public_name and t2.fork = 0
 where not t1.dep_module like 'github.com/%'
 order by 1
  limit 1000


In [60]:
df1 = pd.read_csv("mod-info.oci/name-conv-progress.csv")
df2 = pd.read_csv("module-refs.csv")

df3 = df2.merge(df1, how="left", on="module")
df3.query("github_name == github_name")

,module,refs,github_name,last_updated
0,k8s.io/apimachinery,49117,-,2023-08-25 19:08:22
1,k8s.io/apimachinery,49117,-,2023-08-25 19:09:00
2,k8s.io/apimachinery,49117,-,2023-08-25 19:09:29
3,k8s.io/apimachinery,49117,-,2023-08-25 19:09:29
4,k8s.io/client-go,47051,-,2023-08-25 19:08:23
5,k8s.io/client-go,47051,-,2023-08-25 19:09:00
6,k8s.io/client-go,47051,-,2023-08-25 19:09:29
7,k8s.io/client-go,47051,-,2023-08-25 19:09:30
8,gopkg.in/yaml.v2,45263,-,2023-08-25 19:08:25
9,gopkg.in/yaml.v2,45263,-,2023-08-25 19:09:02


In [50]:
import requests
from html.parser import HTMLParser
    
def convert_name(public_name, parser):
    try:
        print(f"work on {public_name}")
        request_url = f"https://{public_name}"
        resp = requests.get(request_url)
        parser.feed(resp.text)
        return parser.github_name
    except Exception as e:
        print(f"fail to convert {public_name} to github name due to {e}")
    return ""          

class GoImportMetaHTMLParser(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self._github_name = ""
       
    @property
    def github_name(self):
        return self._github_name
         
    def handle_starttag(self, tag, attrs):
        self._gith_name = ""
        if tag == "meta":
            imports_found = [attr for attr in attrs if attr[0] == "name" and attr[1] == "go-import"]
            contents = [attr[1] for attr in attrs if attr[0] == "content"]
            if imports_found and contents:
                comps = contents[0].split(' ')
                idx = comps[2].index("//")
                self._github_name = comps[2] if idx < 0 else comps[2][idx+2:]
                
parser = GoImportMetaHTMLParser()
df['github_name'] = df.apply(lambda row: convert_name(row["module"], parser), axis=1)


In [39]:
%%sql result <<
select dep_module as module,
       count(*) as refs
  from read_csv_auto('dependencies.csv', header=True)
 where dep_module not like 'github.com/%'
   and dep_module not like 'golang.org/%'
 group by dep_module
 order by 2 desc

 -- where module not like 'github.com/%'

Running query in 'duckdb://'

In [54]:
df = result.DataFrame()
df.to_csv("module-refs.csv", index=False)

In [53]:
parser = GoImportMetaHTMLParser()
convert_name("resenje.org/marshal", parser)

work on resenje.org/marshal


''

In [51]:
parser = GoImportMetaHTMLParser()
df['github_name'] = df.apply(lambda row: convert_name(row["module"], parser), axis=1)
df

work on k8s.io/apimachinery
work on k8s.io/client-go
work on gopkg.in/yaml.v2
work on k8s.io/api
work on google.golang.org/grpc
work on google.golang.org/protobuf
work on go.uber.org/zap
work on gopkg.in/yaml.v3
work on sigs.k8s.io/yaml
work on sigs.k8s.io/controller-runtime
work on google.golang.org/genproto
work on k8s.io/utils
work on k8s.io/apiextensions-apiserver
work on google.golang.org/api
work on gorm.io/gorm
work on go.opentelemetry.io/otel
work on k8s.io/klog/v2
work on k8s.io/cli-runtime
work on go.opentelemetry.io/otel/sdk
work on go.opentelemetry.io/otel/trace
work on k8s.io/kubectl
work on k8s.io/apiserver
work on k8s.io/code-generator
work on gotest.tools
work on k8s.io/kube-openapi
work on helm.sh/helm/v3
work on gopkg.in/square/go-jose.v2
work on go.uber.org/atomic
work on k8s.io/component-base
work on go.uber.org/multierr
work on go.mongodb.org/mongo-driver
work on gorm.io/driver/mysql
work on k8s.io/metrics
work on gopkg.in/natefinch/lumberjack.v2
work on k8s.io/klo

KeyboardInterrupt: 

In [33]:
df = result.DataFrame()
df.head(10).to_latex("abc.tex", index=False)

In [15]:
%%sql result <<
select t1.full_name   as full_name,
       t1.public_name as public_name,
       t1.version     as version,
       case when t2.fork = 'False' then 0 else 1 end as fork,
       t1.dep_module  as dep_module,
       t1.dep_version as dep_version
  from read_csv_auto('dependencies.csv', header=True) t1
 inner join read_csv_auto('repo-candidates/slim.csv', header=True) t2
    on t1.full_name = t2.full_name


Running query in 'duckdb://'

In [17]:
df = result.DataFrame()
df.to_csv("dependencies2.csv")

In [ ]:
%%sql
select t1.full_name as dependent_module,
       t1.version as dependent_version,
       t2.module as vul_module,
       t1.dep_version as ref_version,
       t2.fix_version,
       t2.vul_id,
       t2.alias,
       t2.published
  from read_csv_auto('dependencies.csv', header=True) t1
 inner join read_csv_auto('govuls.csv', header=True) t2
    on t2.module = t1.dep_module

In [ ]:
%%sql
select module,
       count(*),
       max(published),
       max(fix_version)
  from read_csv_auto('govuls.csv', header=True)
 group by module
 order by 2 desc